In [3]:
from bs4 import BeautifulSoup
import requests
import re, json
import pandas as pd
import numpy as np
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

In [ ]:
main_page_listing_list = [] # creating list to store search pages of 100 car listings
for i in (range(1)):
    # url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(i * 100) + "&RPG=100"
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(i * 10) + "&RPG=5"
    main_page_listing_list.append(url)


In [ ]:
print(f"Listing list: {main_page_listing_list}")

['https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100']


In [ ]:

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode for faster execution
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [ ]:
base_url_dict = {}
for base_url in main_page_listing_list:
    listing_urls = set()
    driver.get(base_url)
    time.sleep(5)  # Wait for JavaScript to load

    soup = BeautifulSoup(driver.page_source, 'lxml')

    time.sleep(5)

    links = soup.find_all('a', class_='styles_text_link__wBaHL') # Obtaining all vehicle links in the page
    posted_date_div = soup.find_all("div", class_="styles_posted_date__ObxTu") # Obtain the posted date of the vehicle listing

    posted_dates = [posted_date_div.text.strip().replace("Posted", "").strip() for posted_date_div in posted_date_div]

    # Filter out car listing links
    for i in range(len(links)):
        link = links[i]
        href = link['href']
        if 'ID=' in link.get('href') and 'DL=' in link.get('href'):
            full_url = f"https://www.sgcarmart.com{href}" if href.startswith('/') else href
            listing_urls.add(full_url)

    temp_dict = dict(zip(listing_urls, posted_dates))
    base_url_dict[base_url] = temp_dict

# Close the driver
driver.quit()

In [ ]:
len(list(set(listing_urls)))
d = dict(zip(listing_urls, posted_dates))
print(len(d))
print(d)

100
{'https://www.sgcarmart.com/used_cars/info.php?ID=1352281&DL=4189&utm_content=SLeligible': '24-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1368875&DL=4594&utm_content=SLeligible': '24-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1250313&DL=2356&utm_content=SLeligible': '24-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1352519&DL=3116&utm_content=SLeligible': '24-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1352282&DL=4189&utm_content=SLeligible': '24-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1368676&DL=3043&utm_content=SLeligible': '24-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1357928&DL=3538': '24-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1368876&DL=3175&utm_content=SLeligible': '10-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1368771&DL=3758&utm_content=SLeligible': '24-Mar-2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1352547&DL=3758': '24-Ma

## Features Available
### Car Model
### Brand
### Color
### Fuel Type
### Price
### Depreciation
### Registration Date
### COE Left (by extension of Reg Date)
### Mileage
### Manufactured Year
### Road Tax (per year)
### Transmission
### Dereg Value
### OMV
### COE Price
### ARF
### Engine Capacity
### Power
### Curb Weight
### Number of Owners
### Type of Vehicle

In [44]:
import re
import pandas as pd
import numpy as np

def string_extraction(value):
    try:
        num = int(value.replace(',', ''))
    except ValueError:
        num = np.nan  # Use np.nan if you're working with NumPy
    return num   

def get_car_model(json_data):
    """
    Car model data is not automatically displayed on the HTML of the website, 
    hence we have find this from the <script> tag as @context 
    This function utilises information from the <script> tag and in json format data.
    """
    car_name = json_data.get("model")
    return car_name

def get_car_brand(json_data):
    """
    Car Brand data is not automatically displayed on the HTML of the website, 
    hence we have find this from the <script> tag as @context 
    This function utilises information from the <script> tag and in json format data.
    """
    try:
        # Assuming 'json_data' has a 'brand' key with a 'name' key inside
        brand_name = json_data["brand"]["name"]
        return brand_name
    except KeyError:
        # In case of missing keys, return None (this will trigger the fallback)
        return None

def get_car_color(json_data):
    """
    Car color data is not automatically displayed on the HTML of the website, 
    hence we have find this from the <script> tag as @context 
    This function utilises information from the <script> tag and in json format data.
    """
    color = json_data.get("color")
    return color

def get_fuel_type(json_data):
    """
    Fuel type data is not automatically displayed on the HTML of the website, 
    hence we have find this from the <script> tag as @context 
    This function utilises information from the <script> tag and in json format data.
    """
    fuel_type = json_data.get("vehicleEngine")['fuelType']
    return fuel_type

def get_price(json_data):
    """
    Price data is obtainable from the HTML, however to avoid ValueError, IndexError and converting to int,
    We will use Price data from the <script> tag in @context 
    This function utilises information from the <script> tag and in json format data.
    """
    try:
        price = int(json_data["offers"]["price"])
        return price
    except KeyError:
        # In case of missing keys, return None (this will trigger the fallback)
        return None

def get_price_html(url):
    """
    Price data is obtainable from the HTML, this method retrieves price from HTML
    """
    # price = url.find(class_ = 'font_red').text.strip().split('$')
    price = url.find_all(class_='row_bg')[0].find_all('td')[1].text.strip().split(' ')[0].split('$')[1]
    price = string_extraction(price)
    return price

def get_depreciation(url):
    dep = url.find_all(class_='row_bg')[1].find_all('td')[1].text.strip().split(' /')[0].split('$')[1]
    dep = string_extraction(dep)
    return dep

def get_depreciation_soup(soup):
    depreciation_label = soup.find("td", class_="label", string=lambda text: text and "Depreciation" in text)

    # Find the next <td> element, which contains the depreciation value
    if depreciation_label:
        depreciation_value_td = depreciation_label.find_next("td")
        depreciation_text = depreciation_value_td.get_text(strip=True).split('/yr')[0]  # Extract value before "/yr"
        
        # Clean and convert to integer
        depreciation_value = int(depreciation_text.replace("$", "").replace(",", ""))
        
        return depreciation_value
    else:
        return np.nan

def get_reg_date(url):
    reg_date = url.find_all(class_='row_bg')[1].find_all('td')[3].text.strip().split('(')[0]
    return reg_date

def get_mileage(json_data):
    """
    Mileage data is obtainable from the HTML, however to avoid ValueError, IndexError and converting to int,
    We will use Mileage data from the <script> tag in @context 
    This function utilises information from the <script> tag and in json format data.
    """
    try:
        mileage = int(json_data["mileageFromOdometer"]["value"])
        return mileage if mileage != 0 else np.nan  # N.A. mileage is reflected as 0 in scripts, we cannot take 0 as the true mileage
    except (ValueError, KeyError, TypeError):
        return None

def get_mileage_html(url):
    """
    Mileage data is obtainable from the HTML, this method retrieves Mileage from HTML
    """
    try:
        miles = url.find_all(class_='row_info')[0].text.strip().split(' km')[0]
        miles = float(string_extraction(miles))
        return miles
    except (IndexError, ValueError):
        return np.nan

def get_manufactured_year(url):
    """
    Extracts the manufactured year from the given URL's HTML content.
    Handles cases where the required element does not exist.
    """
    try:
        date = url.find_all(class_='row_info')[6].text.strip()
        return int(date)
    except (IndexError, ValueError, AttributeError):
        return None  # Return None if extraction fails

def get_manufactured_year_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        
        for row in rows:
            # Check if the row_title contains "Manufactured"
            if "Manufactured" in row.text:
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    manu_year = row_info.text.strip()
                    manu_year = int(manu_year)
                    return manu_year
    except (AttributeError, ValueError, TypeError):
        return None   # Return np.nan if "Manufactured" is not found

def helper_coe_clean(value):
    days_in_year = 365
    days_in_month = 30

    year_match = re.search(r'(\d+)\s*yrs?', value)
    month_match = re.search(r'(\d+)\s*mths?', value)
    day_match = re.search(r'(\d+)\s*days?', value)

    # Convert to integers (default to 0 if not found)
    years = int(year_match.group(1)) if year_match else 0
    months = int(month_match.group(1)) if month_match else 0
    days = int(day_match.group(1)) if day_match else 0

    return (years * days_in_year) + (months * days_in_month) + days

def get_coe_left(url):
    coe = url.find_all(class_='row_bg')[1].find_all('td')[3].text.strip().split('(')[1].split('COE')[0]
    coe = helper_coe_clean(coe)
    return coe
    
def get_road_tax(url):
    tax = url.find_all(class_='row_info')[1].text.strip().split(' ')[0].split('$')[1]
    tax = string_extraction(tax)
    return tax

def get_road_tax_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        
        for row in rows:
            # Check if the row_title contains "Road Tax"
            if "Road Tax" in row.text:
                # Find the next sibling with class 'row_info'
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    tax_text = row_info.text.strip().split(' ')[0].replace('$', '')
                    return float(tax_text.replace(',', ''))  # Convert to float (handles decimals)
    except (AttributeError, ValueError, TypeError):
        return None  # Return np.nan if "Road Tax" is not found

def get_transmission(url):
    transmission = url.find_all(class_='row_info')[7].text.strip()
    return transmission

def get_transmission_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        for row in rows:
            if "Transmission" in row.text:
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    return row_info.text.strip()
    except (AttributeError, ValueError, TypeError):
        return None # Return np.nan if "Transmission" is not found

def get_dereg_value(url):
    try:
        dereg_val = url.find_all(class_='row_info')[2].text.strip().split(' as')[0].split('$')[1]
        dereg_val = string_extraction(dereg_val)
    except:
        dereg_val = np.nan
    return dereg_val

def get_dereg_value_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        for row in rows:
            if "Dereg Value" in row.text:
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    dereg = row_info.text.strip().split(' as')[0].split('$')[1]
                    dereg = float(string_extraction(dereg))
                    return dereg
    except (AttributeError, ValueError, TypeError):   
        return None  # Return np.nan if "Dereg Value" is not found

def get_omv(url):
    omv = url.find_all(class_='row_info')[8].text.strip().split('$')[1]
    omv = string_extraction(omv)
    return omv

def get_omv_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        for row in rows:
            if "OMV" in row.text:
                # Find the next sibling with class 'row_info'
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    omv = row_info.text.strip().split('$')[1]
                    omv = float(string_extraction(omv))
                    return omv
    except (AttributeError, ValueError, TypeError):   
        return None  # Return np.nan if "OMV" is not found

def get_coe_value(url):
    try:
        coe_val = url.find_all(class_='row_info')[3].text.strip().split('$')[1]
        coe_val = string_extraction(coe_val)
    except:
        coe_val = np.nan
    return coe_val

def get_coe_value_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        for row in rows:
            if "COE" in row.text:
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    coe = row_info.text.strip().split('$')[1]
                    coe = string_extraction(coe)
                    return coe
    except (AttributeError, ValueError, TypeError):   
        return None  # Return np.nan if "COE" is not found

def get_arf(url):
    arf_val = url.find_all(class_='row_info')[9].text.strip().split('$')[1]
    arf_val = string_extraction(arf_val)
    return arf_val

def get_arf_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        for row in rows:
            if "ARF" in row.text:
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    arf = row_info.text.strip().split('$')[1]
                    arf = string_extraction(arf)
                return arf
    except (AttributeError, ValueError, TypeError):   
        return None  # Return np.nan if "ARF" is not found

def get_engine_capacity(url):
    engine_cap = url.find_all(class_='row_info')[4].text.strip().split(' cc')[0]
    engine_cap = string_extraction(engine_cap)
    return engine_cap

def get_power(url):
    try:
        power_text = url.find_all(class_='row_info')[10].text.strip().split(' kW')[0]
        power = float(power_text)
        return power
    except (IndexError, ValueError):
        return np.nan

def get_power_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        for row in rows:
            if "Power" in row.text:
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    power_text = row_info.text.strip().split(' kW')[0]
                    try:
                        power = float(power_text)
                        return power
                    except ValueError:
                        return np.nan  # If conversion fails, return NaN
    except (AttributeError, ValueError, TypeError):   
        return None  # Return np.nan if "Power" is not found

def get_curb_weight(url):
    curb_weight = string_extraction(url.find_all(class_='row_info')[5].text.strip().split(' kg')[0])
    return curb_weight

def get_curb_weight_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        for row in rows:
            if "Curb Weight" in row.text:
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    weight_text = row_info.text.strip().split(' kg')[0]
                    weight = string_extraction(weight_text)
                    return weight
    except (AttributeError, ValueError, TypeError):   
        return None  # Return np.nan if "Curb Weight" is not found

def get_number_of_owners(url):
    owner_text = url.find_all(class_='row_info')[11].text.strip()
    
    # Extract numbers using regex
    numbers = re.findall(r'\d+', owner_text)
    
    if not numbers:
        return np.nan 
    
    owner_num = int(numbers[0]) 

    # SGCarMart uses "less than" and "more than" to indicate ranges of owners
    if "less than" in owner_text.lower():
        owner_num -= 1  # Assume it's at most owner_num - 1
    elif "more than" in owner_text.lower():
        owner_num += 1  # Assume it's at least owner_num + 1

    return owner_num

def get_number_of_owners_soup(soup):
    try:
        rows = soup.find_all(class_='row_title')
        for row in rows:
            if "Owners" in row.text:
                row_info = row.find_next_sibling(class_='row_info')
                if row_info:
                    owner_text = row_info.text.strip()
                    numbers = re.findall(r'\d+', owner_text)
                    if not numbers:
                        return np.nan
                    owner_num = int(numbers[0])
                    if "less than" in owner_text.lower():
                        owner_num -= 1
                    elif "more than" in owner_text.lower():
                        owner_num += 1
                    return owner_num
    except (AttributeError, ValueError, TypeError):   
        return None  # Return np.nan if "Owners" is not found

def get_type_of_vehicle(json_data):
    try:
        types = json_data.get("bodyType")
        return types
    except KeyError:
        # In case of missing keys, return None (this will trigger the fallback)
        return None

def get_type_of_vehicle_html(url):
    types = url.find_all(class_='row_bg1')[0].find('a').text.strip()
    return types

def get_posted_date(soup):
    """Extracts the posted date from the HTML."""
    post_date_div = soup.find("div", id="usedcar_postdate")
    if post_date_div:
        parts = post_date_div.text.split("|")  # Split by the separator
        if len(parts) > 0:
            return parts[0].replace("Posted on:", "").strip()  # Extract posted date
    return None

def get_last_updated_date(soup):
    """Extracts only the date from the Last Updated text in the HTML."""
    post_date_div = soup.find("div", id="usedcar_postdate")
    if post_date_div:
        match = re.search(r"Last\s*Updated\s*on:\s*([\d\-A-Za-z]+)", post_date_div.text)
        if match:
            return match.group(1)  # Extracts only the date
    return None

def safe_extract(func, *args, default=np.nan):
    """Utility function to safely extract values with a fallback."""
    try:
        return func(*args)
    except:
        return default

In [39]:
# a = requests.get('https://www.sgcarmart.com/used_cars/info.php?ID=1372995&DL=3730&GASRC=dy')
# a = requests.get('https://www.sgcarmart.com/used_cars/info.php?ID=1343041&DL=1220&GASRC=dy')
a = requests.get('https://www.sgcarmart.com/used_cars/info.php?ID=1384592&DL=4436&GASRC=sgcm')
b = BeautifulSoup(a.text, 'lxml')

# get_price(b)
get_last_updated_date(b)
# depreciation(b)
# mileage(b)

'27-Mar-2025'

In [40]:
## Features Available
### Car Model
### Brand
### Color
### Fuel Type
### Price
### Depreciation
### Registration Date
### COE Left (by extension of Reg Date)
### Mileage
### Manufactured Year
### Road Tax (per year)
### Transmission
### Dereg Value
### OMV
### COE Value
### ARF
### Engine Capacity
### Power
### Curb Weight
### Number of Owners
### Type of Vehicle
### Date Posted
### Last Updated

df = pd.DataFrame(columns=['listing_url', 'car_model','brand', 'color', 'fuel_type', 'price',
                           'depreciation_per_year', 'registration_date', 'coe_left', 'mileage', 'manufactured_year',
                           'road_tax_per_year', 'transmission', 'dereg_value', 'omv', 'coe_value', 'arf',
                            'engine_capacity_cc', 'power', 'curb_weight', 'no_of_owners', 'vehicle_type', 'date_posted', 'last_updated'])
       # 'REG_DATE', 'MILEAGE/KM', 'MANUFACTURED_YEAR',
       # 'ROAD_TAX_PER_YEAR','TRANSMISSION', 'DEREG_VALUE_FROM_SCRAPE_DATE',
       # 'SCRAPE_DATE', 'OMV', 'ARF', 'COE_FROM_SCRAPE_DATE',
       # 'DAYS_OF_COE_LEFT', 'ENGINE_CAPACITY_CC', 
       # 'NO_OF_OWNERS', 'VEHICLE_TYPE'])
file = 'singapore_car_dateset.csv'

In [46]:


i = 0 

# for listingurl in listing_urls:
#     print(listingurl)
listingurl='https://www.sgcarmart.com/used_cars/info.php?ID=1387503&DL=1000&utm_content=SLeligible'
# listingurl = 'https://www.sgcarmart.com/used_cars/info.php?ID=1387666&DL=4189&utm_content=SLeligible'
response = requests.get(listingurl)

listing_url = BeautifulSoup(response.text, 'lxml')
soup = BeautifulSoup(response.text, "html.parser")
# time.sleep(5)  # Wait for JavaScript to load
script_tag = soup.find("script", string=re.compile(r'@context'))
json_data = np.nan

# If script_tag with @context is found, parse its content
if script_tag:
    try:
        json_text = script_tag.string.strip()
        json_data = json.loads(json_text)
    except json.JSONDecodeError:
        pass  # If it can't decode, leave json_data as NaN

# If no @context script tag is found, check for window._loopaData as fallback
if json_data is np.nan:
    script_tag = soup.find("script", text=lambda t: t and 'window._loopaData' in t)
    if script_tag:
        try:
            # Extract the JSON from the window._loopaData assignment
            json_text = script_tag.string.split('window._loopaData = ')[1].split(';')[0]
            json_data = json.loads(json_text)
        except json.JSONDecodeError:
            pass  # If it can't decode, leave json_data as NaN

# Extract details using JSON if available; otherwise, fallback to HTML parsing
car_model = safe_extract(get_car_model, json_data)
brand_name = safe_extract(get_car_brand, json_data) or json_data.get('make')
color = safe_extract(get_car_color, json_data)
fuel_type = safe_extract(get_fuel_type, json_data)
price = safe_extract(get_price, json_data) or safe_extract(get_price_html, listing_url)
depreciation = safe_extract(get_depreciation, listing_url)
reg_date = safe_extract(get_reg_date, listing_url)
coe_remaining = safe_extract(get_coe_left, listing_url)
mileage = safe_extract(get_mileage, json_data) or safe_extract(get_mileage_html, listing_url)
manu_year = safe_extract(get_manufactured_year_soup, soup) or safe_extract(get_manufactured_year, listing_url) 
road_tax = safe_extract(get_road_tax_soup, soup) or safe_extract(get_road_tax, listing_url)
transmission = safe_extract(get_transmission_soup, soup) or safe_extract(get_transmission, listing_url)
dereg_value = safe_extract(get_dereg_value_soup, soup) or safe_extract(get_dereg_value, listing_url)
omv = safe_extract(get_omv_soup, soup) or safe_extract(get_omv, listing_url)
coe_value = safe_extract(get_coe_value_soup, soup) or safe_extract(get_coe_value, listing_url)
arf = safe_extract(get_arf_soup, soup) or safe_extract(get_arf, listing_url) 
engine_capacity = np.nan if fuel_type == "Electric" else safe_extract(get_engine_capacity, listing_url)
power = safe_extract(get_power_soup, soup) or safe_extract(get_power, listing_url)
curb_weight = safe_extract(get_curb_weight_soup, soup) or safe_extract(get_curb_weight, listing_url)
no_of_owners = safe_extract(get_number_of_owners_soup, soup) or safe_extract(get_number_of_owners, listing_url)
vehicle_type = safe_extract(get_type_of_vehicle, json_data) or safe_extract(get_type_of_vehicle_html, listing_url)
date_posted = safe_extract(get_posted_date, soup)
last_updated = safe_extract(get_last_updated_date, soup)

print(f"Car Model: {car_model}")
print(f"Brand: {brand_name}")
print(f"Color: {color}")
print(f"Fuel Type: {fuel_type}")
print(f"Price: {price} SGD")
print(f"Depreciation: {depreciation} SGD")
print(f"Registration Date: {reg_date}")
print(f"COE Left: {coe_remaining} days")
print(f"Mileage: {mileage} km")
print(f"Manufactured: {manu_year}")
print(f"Road Tax: {road_tax} SGD")
print(f"Transmission: {transmission}")
print(f"Dereg Value: {dereg_value} SGD")
print(f"OMV: {omv} SGD")
print(f"COE Value: {coe_value} SGD")
print(f"ARF: {arf} SGD")
print(f"Engine Capacity: {engine_capacity} cc")
print(f"Power: {power} kW")
print(f"Curb Weight: {curb_weight} kg")
print(f"No. of Owners: {no_of_owners}")
print(f"Vehicle Type: {vehicle_type}")
print(f"Date Posted: {date_posted}")
print(f"Last Updated: {last_updated}")


# #df = pd.DataFrame(columns=['listing_url', 'car_model','brand', 'color', 'fuel_type', 'price',
#                        'depreciation_per_year', 'registration_date', 'coe_left', 'mileage', 'manufactured_year',
#                        'road_tax_per_year', 'transmission', 'dereg_value', 'omv', 'coe_value', 'arf',
#                         'engine_capacity_cc', 'power', 'curb_weight', 'no_of_owners', 'vehicle_type'])
df.loc[i, 'listing_url'] = listingurl
df.loc[i, 'car_model'] = car_model
df.loc[i, 'brand'] = brand_name
df.loc[i, 'color'] = color
df.loc[i, 'fuel_type'] = fuel_type
df.loc[i, 'price'] = price
df.loc[i, 'depreciation_per_year'] = depreciation
df.loc[i, 'registration_date'] = reg_date
df.loc[i, 'coe_left'] = coe_remaining
df.loc[i, 'mileage'] = mileage
df.loc[i, 'manufactured_year'] = manu_year
df.loc[i, 'road_tax_per_year'] = road_tax
df.loc[i, 'transmission'] = transmission
df.loc[i, 'dereg_value'] = dereg_value
df.loc[i, 'omv'] = omv
df.loc[i, 'coe_value'] = coe_value
df.loc[i, 'arf'] = arf
df.loc[i, 'engine_capacity_cc'] = engine_capacity
df.loc[i, 'power'] = power
df.loc[i, 'curb_weight'] = curb_weight
df.loc[i, 'no_of_owners'] = no_of_owners
df.loc[i, 'vehicle_type'] = vehicle_type
df.loc[i, 'date_posted'] = date_posted
df.loc[i, 'last_updated'] = last_updated

# df.to_csv(file, index=False)
    # i += 1 # Allows next car listing to be put into a next row in the dataframe
    # time.sleep(2)  # Prevents us from getting locked out of the website

Car Model: Avante
Brand: Hyundai
Color: None
Fuel Type: nan
Price: 115888 SGD
Depreciation: 15700 SGD
Registration Date: 18-May-2022
COE Left: 2602 days
Mileage: 82000.0 km
Manufactured: 2021
Road Tax: 742.0 SGD
Transmission: Auto
Dereg Value: 56354.0 SGD
OMV: 21906.0 SGD
COE Value: 70901 SGD
ARF: 7669 SGD
Engine Capacity: 1598 cc
Power: 90.2 kW
Curb Weight: 1280 kg
No. of Owners: 1
Vehicle Type: Mid-Sized Sedan
Date Posted: 30-Mar-2025
Last Updated: 30-Mar-2025


C:\Users\joel-\AppData\Local\Temp\ipykernel_8208\1178534135.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup.find("script", text=lambda t: t and 'window._loopaData' in t)


In [6]:
main_page_listing_list = [] # creating list to store search pages of 100 car listings
for i in (range(1)):
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(i * 100) + "&RPG=100"
main_page_listing_list.append(url)

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode for faster execution
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url_dict = {}
for base_url in main_page_listing_list:
    listing_urls = set()
    driver.get(base_url)
    print(base_url)
    time.sleep(5)  # Wait for JavaScript to load

    soup = BeautifulSoup(driver.page_source, 'lxml')

    links = soup.find_all('a', class_='styles_text_link__wBaHL') # Obtaining all vehicle links in the page
    posted_date_div = soup.find_all("div", class_="styles_posted_date__ObxTu") # Obtain the posted date of the vehicle listing

    posted_dates = [posted_date_div.text.strip().replace("Posted", "").strip() for posted_date_div in posted_date_div]
    print(links)
    print(len(links))
    print(len(posted_dates))

    # Filter out car listing links
    for i in range(len(links)):
        link = links[i]
        href = link['href']
        if 'ID=' in link.get('href') and 'DL=' in link.get('href'):
            full_url = f"https://www.sgcarmart.com{href}" if href.startswith('/') else href
            listing_urls.add(full_url)

    temp_dict = dict(zip(listing_urls, posted_dates))
    base_url_dict[base_url] = temp_dict
    # print(base_url_dict)
    # print

# Close the driver
driver.quit()

print(base_url_dict) # {base_url_1: {vehicle_link_1: posted_date_1, ...}, ...}
# print(len(base_url_dict))

https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100
[<a class="styles_text_link__wBaHL" href="https://www.sgcarmart.com/used_cars/info.php?ID=1385611&amp;DL=536"><div class="styles_model_name__ZaHTI">Hyundai Avante 1.6A GLS S</div></a>, <a class="styles_text_link__wBaHL" href="https://www.sgcarmart.com/used_cars/info.php?ID=1385611&amp;DL=536"><div class="styles_flex_content_box__RA7Ow"><div class="styles_left_container__Qh_1c"><div class="styles_image_container__hmoZb"><div class="styles_tagOverlayContainer__A2XHA"><img alt="Hyundai Avante 1.6A GLS S" class="styles_image__FrS3i styles_listing_Mimage__1E8xK" data-nimg="1" decoding="async" height="100" loading="lazy" src="https://i.i-sgcm.com/cars_used/202503/1385611_1.jpg" style="color: transparent;" width="133"/></div></div><div class="styles_car_tag__eEfFc">Auction Vehicle</div></div><div class="styles_right_container__Sphjv"><div class="styles_price_container__rI4oV listing_price_box__AykqY"><div class="styles_price__PoUI

In [167]:
df.to_csv(file,index=False)

In [ ]:

df['LIS']

,LISTING_URL,BRAND,PRICE,DEPRE_VALUE_PER_YEAR,REG_DATE,MILEAGE/KM,MANUFACTURED_YEAR,ROAD_TAX_PER_YEAR,TRANSMISSION,DEREG_VALUE_FROM_SCRAPE_DATE,SCRAPE_DATE,OMV,ARF,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,NO_OF_OWNERS,VEHICLE_TYPE,MILEAGE_KM
0,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,30800,6270,18-May-2010,NaN,2010.0,742.0,Auto,16141.0,20/06/2020,16716.0,16716.0,16438.0,1467.0,1598.0,3.0,Mid-Sized Sedan,140000.0
1,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,NaN,NaN,07-Oct-2009,NaN,2009.0,NaN,Auto,NaN,20/06/2020,39714.0,1986.0,12916.0,1550.0,2982.0,NaN,Van,NaN
2,https://www.sgcarmart.com/used_cars/info.php?I...,BMW,92500,10590,26-Sep-2017,NaN,2017.0,684.0,Auto,49443.0,20/06/2020,27881.0,31034.0,36001.0,2650.0,1499.0,2.0,MPV,15500.0
3,https://www.sgcarmart.com/used_cars/info.php?I...,Volkswagen,NaN,NaN,30-Nov-2018,NaN,2018.0,976.0,Auto,NaN,20/06/2020,32136.0,36991.0,32000.0,3079.0,1798.0,1.0,Luxury Sedan,NaN
4,https://www.sgcarmart.com/used_cars/info.php?I...,Mercedes-Benz,NaN,NaN,24-Mar-2017,NaN,2017.0,NaN,Manual,NaN,20/06/2020,27595.0,1380.0,46302.0,2463.0,1598.0,NaN,Van,NaN
5,https://www.sgcarmart.com/used_cars/info.php?I...,Porsche,318888,NaN,08-Oct-2010,NaN,2009.0,3632.0,Manual,82683.0,20/06/2020,162778.0,162778.0,43334.0,NaN,3797.0,3.0,Sports Car,31888.0
6,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,NaN,NaN,27-Jul-2006,NaN,2006.0,1361.0,Auto,NaN,20/06/2020,19525.0,21478.0,25815.0,401.0,1794.0,2.0,MPV,NaN
7,https://www.sgcarmart.com/used_cars/info.php?I...,BMW,52588,6350,25-Nov-2008,NaN,2008.0,1450.0,Auto,27653.0,20/06/2020,39080.0,39080.0,33377.0,3010.0,1995.0,3.0,Luxury Sedan,103580.0
8,https://www.sgcarmart.com/used_cars/info.php?I...,Daihatsu,26800,5360,18-Mar-2010,NaN,2009.0,566.0,Auto,NaN,20/06/2020,15061.0,15061.0,NaN,NaN,1298.0,3.0,Hatchback,96000.0
9,https://www.sgcarmart.com/used_cars/info.php?I...,Subaru,57800,9330,11-Nov-2015,NaN,2015.0,1208.0,Auto,42558.0,20/06/2020,14948.0,14948.0,58109.0,1946.0,1995.0,1.0,SUV,80644.0


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'C:\Users\joel-\Downloads\vehicle_details_0_20250330_103042.csv')
columns_to_convert = ['used_car_id', 'coe_left', 'mileage', 
                                          'manufactured_year', 'engine_capacity_cc', 
                                          'power', 'curb_weight', 'no_of_owners']
for col in columns_to_convert:
    df[col] = np.floor(pd.to_numeric(df[col], errors='coerce')).astype("Int64")


# Convert the following columns to FLOAT type for BigQuery
df['price'] = df['price'].astype("float")
df['depreciation_per_year'] = df['depreciation_per_year'].astype("float")
df['road_tax_per_year'] = df['road_tax_per_year'].astype("float")
df['dereg_value'] = df['dereg_value'].astype("float")
df['omv'] = df['omv'].astype("float")
df['coe_value'] = df['coe_value'].astype("float")
df['arf'] = df['arf'].astype("float")

# Convert the following columns to DATE type for BigQuery
df['registration_date'] = pd.to_datetime(df['registration_date'], format="%Y-%m-%d").dt.date
df['posted_datetime'] = pd.to_datetime(df['posted_datetime'], format="%Y-%m-%d").dt.date
df['updated_datetime'] = pd.to_datetime(df['updated_datetime'], format="%Y-%m-%d").dt.date
df['scraped_datetime'] = pd.to_datetime(df['scraped_datetime'], format="%Y-%m-%d").dt.date

print(df.head())
# Check the type of each column
print(df.dtypes)
# Check the type of each cell in the first row
print(df.iloc[0].apply(type))

   used_car_id                                        listing_url  \
0      1350157  https://www.sgcarmart.com/used_cars/info.php?I...   
1      1383530  https://www.sgcarmart.com/used_cars/info.php?I...   
2      1387656  https://www.sgcarmart.com/used_cars/info.php?I...   
3      1353352  https://www.sgcarmart.com/used_cars/info.php?I...   
4      1387657  https://www.sgcarmart.com/used_cars/info.php?I...   

                           car_model      brand  color fuel_type    price  \
0                                NaN        NaN    NaN       NaN      NaN   
1                    M.G. HS 1.5T AT       M.G.  Black    Petrol  88300.0   
2  SUBARU FORESTER 2.0I-L CVT AWD SR     SUBARU  Brown    Petrol  48800.0   
3                                NaN        NaN    NaN       NaN      NaN   
4            CHEVROLET CRUZE SW J308  CHEVROLET    Red    Petrol  23700.0   

   depreciation_per_year registration_date  coe_left  ...  engine_capacity_cc  \
0                11110.0        2023-05-0